In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [4]:
from cancerClassification.constants import *
from cancerClassification.utils.common import read_yaml, create_directories
import tensorflow as tf

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

        

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chest-CT-Scan-data")
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [6]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [7]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )
    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
    
    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [8]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
except Exception as e:
    raise e

[2024-10-01 13:44:38,395: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-10-01 13:44:38,396: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-01 13:44:38,397: INFO: common: created directory at: artifacts]
[2024-10-01 13:44:38,398: INFO: common: created directory at: artifacts/training]
Found 68 images belonging to 2 classes.
Found 275 images belonging to 2 classes.
Epoch 1/100


2024-10-01 13:44:38.897836: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


27/27 [==============================] - 5s 133ms/step - loss: 17.5923 - accuracy: 0.4943 - val_loss: 1.1715 - val_accuracy: 0.8667
Epoch 2/100
27/27 [==============================] - 3s 125ms/step - loss: 12.5541 - accuracy: 0.5811 - val_loss: 5.2058 - val_accuracy: 0.6500
Epoch 3/100
27/27 [==============================] - 3s 125ms/step - loss: 6.5407 - accuracy: 0.6868 - val_loss: 1.9799 - val_accuracy: 0.8667
Epoch 4/100
27/27 [==============================] - 3s 124ms/step - loss: 4.7603 - accuracy: 0.7585 - val_loss: 0.0465 - val_accuracy: 0.9833
Epoch 5/100
27/27 [==============================] - 3s 125ms/step - loss: 1.7151 - accuracy: 0.8453 - val_loss: 0.1148 - val_accuracy: 0.9000
Epoch 6/100
27/27 [==============================] - 3s 124ms/step - loss: 3.3977 - accuracy: 0.7887 - val_loss: 5.2006e-04 - val_accuracy: 1.0000
Epoch 7/100
27/27 [==============================] - 3s 123ms/step - loss: 1.8139 - accuracy: 0.8453 - val_loss: 3.0331 - val_accuracy: 0.8667
Epoch